#  `tf.GradientTape` 自动求导的核心工具的基本用法
## 核心概念：tf.GradientTape 是什么？
你可以把 `tf.GradientTape` 想象成一个“磁带录音机”。

录制 (Recording)：当你开启一个 `GradientTape` 的上下文环境时，它会像录音机一样，自动“录制”所有在 TensorFlow 中执行的、涉及可训练变量 (`tf.Variable`) 的计算步骤。

 - 回放 (Rewinding)：计算完成后，你可以调用 `.gradient()` 方法，`GradientTape` 会“回放”这个磁带（在数学上是利用反向传播和链式法则），计算出某个结果（通常是损失函数）相对于一个或多个变量的梯度。

 - 销毁 (Erasing)：默认情况下，一旦 `.gradient()` 方法被调用，这个“磁带”上的内容就会被立即释放，以节省内存。

这是实现自动微分（Automatic Differentiation）的关键，也是训练神经网络的基础。

---
## 基本用法三步曲
使用 `tf.GradientTape` 通常遵循以下三个步骤：

### 1. 创建 `GradientTape` 上下文
使用 Python 的 `with` 语句来创建一个录制环境。所有在这个 `with` 代码块中进行的计算都会被追踪。

In [ ]:
import tensorflow as tf

# 准备一些变量
# tf.Variable 是可训练的，会被 GradientTape 自动追踪
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
  # 在 with 代码块内定义需要求导的计算
  y = x * x # 等价于 x**2

### 2. 定义计算过程
在 `with` 代码块内部，定义你的`前向传播计算`。这可以是一个简单的数学公式，也可以是一个复杂的神经网络模型。`GradientTape` 会自动监控所有 `tf.Variable` 并记录相关的操作。

在上面的例子中，y = x * x 就是我们关心的计算过程。

---
### 3. 计算梯度
使用 `tape.gradient(target, sources)` 方法来计算梯度。

 - `target`: 你想要对其求导的目标，通常是模型的损失函数（`loss`），必须是一个标量（`scalar`）。

 - `sources`: 你想要计算梯度的来源，通常是模型的权重（`tf.Variable`），可以是一个变量，也可以是一个变量列表。

In [ ]:
# 计算 y 相对于 x 的梯度
# target 是 y, source 是 x
dy_dx = tape.gradient(y, x)

print(dy_dx)

输出结果：
tf.Tensor(6.0, shape=(), dtype=float32)

这个结果是完全正确的，因为函数 y=x*x 的导数是 2x。当 x=3.0 时，导数就是 $2 * 3.0 = 6.0$。

---
## 关键参数和进阶用法
### 1. 追踪 `tf.Tensor`：tape.watch()
默认情况下，`GradientTape` 只会自动追踪 `tf.Variable`。如果你想对一个普通的 `tf.Tensor` 求梯度，你需要手动告诉 "录音机" 开始监视这个张量。

In [ ]:
x = tf.constant(3.0) # 这是一个常量 Tensor，不是 Variable

with tf.GradientTape() as tape:
  # 必须手动 watch 一个非 Variable 的 Tensor
  tape.watch(x)
  y = x * x

dy_dx = tape.gradient(y, x)
print(dy_dx) # tf.Tensor(6.0, shape=(), dtype=float32)

注意：在构建模型时，模型的`权重`通常都定义为 `tf.Variable`，所以你很少需要手动 `watch`。

---
### 2. 计算多个梯度：persistent=True
前面提到，磁带在调用 `.gradient()` 后会立即被擦除。如果你需要对同一个计算过程调用多次` .gradient()`（例如，计算一个复杂的损失函数对不同层权重的梯度），你需要让磁带“持久化”。使用完持久化的 `tape` 后，手动删除它以释放资源 `del tape`

In [ ]:
x = tf.Variable(4.0)
y = tf.Variable(3.0)

with tf.GradientTape(persistent=True) as tape:
  z1 = x * x
  z2 = y * y
  z = z1 + z2

# 计算 z 相对于 x 的梯度 (∂z/∂x = 2x = 8.0)
dz_dx = tape.gradient(z, x)
print(dz_dx)

# 计算 z 相对于 y 的梯度 (∂z/∂y = 2y = 6.0)
# 如果没有 persistent=True, 这一步会报错！
dz_dy = tape.gradient(z, y)
print(dz_dy)

# 使用完持久化的 tape 后，手动删除它以释放资源
del tape

### 3. 梯度为 None 的常见情况
有时 `tape.gradient()` 会返回 `None`，这通常意味着：

 - 目标和来源之间没有计算路径：你求导的变量没有参与目标的计算。

 - 变量是整数类型：梯度只能对浮点数类型（如 `float32`, `float64`）的变量计算。

 - 计算在 `TensorFlow` 之外进行：例如，你使用了 NumPy 的操作，`GradientTape` 无法追踪到它。

 - 变量被放置在` with tape `代码块之外创建或计算。

In [ ]:
x = tf.Variable(2.0)
y1 = x * x # y1 的计算在 tape 外部，未被录制

with tf.GradientTape() as tape:
  y2 = x + 1 # y2 的计算在 tape 内部，被录制

print(tape.gradient(y1, x)) # 输出 None，因为 y1 的计算未被追踪
print(tape.gradient(y2, x)) # 输出 tf.Tensor(1.0, ...)，因为 y2 的计算被追踪

## 综合实例：一个简单的线性回归模型
下面是一个完整的例子，展示了如何使用 `tf.GradientTape` 来训练一个简单的模型 y=Wx+b。

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 1. 准备数据
X_raw = np.array([3.3, 4.4, 5.5, 6.71, 6.93, 4.168, 9.779, 6.182, 7.59, 2.167,
              7.042, 10.791, 5.313, 7.997, 5.654, 9.27, 3.1], dtype=np.float32)
y_raw = np.array([1.7, 2.76, 2.09, 3.19, 1.694, 1.573, 3.366, 2.596, 2.53, 1.221,
              2.827, 3.465, 1.65, 2.904, 2.42, 2.94, 1.3], dtype=np.float32)

X = tf.constant(X_raw)
y_true = tf.constant(y_raw)

# 2. 初始化模型参数 (必须是 tf.Variable)
W = tf.Variable(tf.random.normal([1]), name="weight")
b = tf.Variable(tf.zeros([1]), name="bias")

learning_rate = 0.01
epochs = 100

# 3. 训练循环
for epoch in range(epochs):
  # 开启 GradientTape
  with tf.GradientTape() as tape:
    # 前向传播：计算预测值
    y_pred = W * X + b
    # 计算损失 (均方误差)
    loss = tf.reduce_mean(tf.square(y_pred - y_true))

  # 计算损失函数对 W 和 b 的梯度
  # tape.gradient(目标, [来源列表])
  gradients = tape.gradient(loss, [W, b])
  dW, db = gradients[0], gradients[1]

  # 反向传播：更新权重和偏置
  # W = W - learning_rate * dW
  W.assign_sub(learning_rate * dW)
  b.assign_sub(learning_rate * db)

  if (epoch + 1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.numpy():.4f}, W: {W.numpy()[0]:.4f}, b: {b.numpy()[0]:.4f}')

print("\n训练完成!")
print(f"最终模型: y = {W.numpy()[0]:.4f}x + {b.numpy()[0]:.4f}")

# 可视化结果
plt.scatter(X, y_true, label='Original data')
plt.plot(X, W * X + b, 'r-', label='Fitted line')
plt.legend()
plt.show()

在这个例子中，`tf.GradientTape` 录制了从 `W `和` b `到 `loss` 的整个计算图，然后我们用 `tape.gradient()` 得到了损失对参数的梯度，并用这些梯度来更新参数，从而使模型的预测越来越准。这就是机器学习训练的核心循环。